In [5]:
import scipy as sp
import scipy.sparse as sps
import numpy as np
import torch
import xitorch
from xitorch import linalg
from scipy.sparse import csr_matrix

EJ = torch.tensor(10.00, requires_grad=False, dtype=torch.double)
EL = torch.tensor(0.04, requires_grad=False, dtype=torch.double)
ECs = torch.tensor(20, requires_grad=False, dtype=torch.double)
EC = torch.tensor(0.04, requires_grad=False, dtype=torch.double)
dEj = torch.tensor(0.0, requires_grad=False, dtype=torch.double)
dCj = torch.tensor(0.0, requires_grad=False, dtype=torch.double)
ECj =  torch.tensor(0.2, requires_grad=False, dtype=torch.double)
flux = torch.tensor(0.5, requires_grad=False, dtype=torch.double)

ng = 0.1
phi_ext = 0.5
varphi_ext =0.5

Nphi = 100
Ntheta = 100

In [6]:
eye_Nphi = torch.eye(Nphi)
eye_Ntheta = torch.eye(Ntheta)

partial_phi_fd = torch.kron(eye_Ntheta, torch.tensor(sps.diags([-1, 1, 1], [0, 1, -Nphi+1], shape=(Nphi, Nphi)).todense()))
partial_phi_bk = torch.kron(eye_Ntheta, torch.tensor(sps.diags([1, -1, -1], [0, -1, Nphi-1], shape=(Nphi, Nphi)).todense()))
partial_theta_fd = torch.kron(eye_Nphi, torch.tensor(sps.diags([-1, 1, 1], [0, 1, -Ntheta+1], shape=(Ntheta, Ntheta)).todense()))
partial_theta_bk =torch.kron(eye_Nphi, torch.tensor(sps.diags([1, -1, -1], [0, -1, Ntheta-1], shape=(Ntheta, Ntheta)).todense()))


phi = np.linspace(0, 2 * np.pi, Nphi)
cos_phi = np.cos(phi)
cos_phi_m = np.diag(cos_phi)
sin_phi_adj = np.sin(phi-phi_ext/2)
sin_phi_adj_m = np.diag(sin_phi_adj)
phi_m = np.diag(phi)
_cos_phi = torch.kron(torch.tensor(cos_phi_m), torch.tensor(eye_Nphi) )
_phi = torch.kron(torch.tensor(phi_m), torch.tensor(eye_Nphi) )
_sin_phi_adj_m  = torch.kron(torch.tensor(sin_phi_adj_m), torch.tensor(eye_Nphi))

theta = np.linspace(0, 2 * np.pi, Ntheta)
cos_theta_adj = np.cos(theta-varphi_ext/2)
sin_theta = np.sin(theta)
cos_theta_adj_m = np.diag(cos_theta_adj)
sin_theta_m = np.diag(sin_theta)
_cos_theta_adj_m = torch.kron(torch.tensor(cos_theta_adj_m), torch.tensor(eye_Ntheta))
_sin_theta_m = torch.kron(torch.tensor(sin_theta_m), torch.tensor(eye_Ntheta))

I = torch.kron(torch.tensor(eye_Nphi), torch.tensor(eye_Ntheta))

Ham = -2 * ECj * (partial_phi_fd * partial_phi_bk) \
    + 2 * ECs * (-1* partial_theta_fd**2 +ng**2*I-2*ng*partial_theta_fd)\
    + 2 * ECs * dCj * partial_phi_fd * partial_theta_fd \
    - 2 * EJ * _cos_phi * _cos_theta_adj_m \
    + EL * _phi ** 2 \
    + 2 * EJ * I  \
    + EJ * dEj * _sin_theta_m * _sin_phi_adj_m

H = Ham + torch.transpose(Ham, 1,0 )

/var/folders/8k/9jgfk1f10xd019kt_x6h66yc0000gn/T/ipykernel_43728/2030465895.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _cos_phi = torch.kron(torch.tensor(cos_phi_m), torch.tensor(eye_Nphi) )
/var/folders/8k/9jgfk1f10xd019kt_x6h66yc0000gn/T/ipykernel_43728/2030465895.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _phi = torch.kron(torch.tensor(phi_m), torch.tensor(eye_Nphi) )
/var/folders/8k/9jgfk1f10xd019kt_x6h66yc0000gn/T/ipykernel_43728/2030465895.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _sin_phi_adj_m  = tor

In [7]:
H_numpy = H.numpy()
H_sparse = H.to_sparse()

def pytorch_sparse_to_numpy(tensor):
    # Ensure the tensor is sparse
    assert tensor.is_sparse, "Input should be a sparse tensor"

    # Get the tensor attributes
    indices = tensor._indices().numpy()
    values = tensor._values().numpy()
    size = tensor.size()

    # Construct the corresponding SciPy sparse matrix
    return csr_matrix((values, indices), shape=size)

H_sparse_numpy = pytorch_sparse_to_numpy(H_sparse)

In [8]:
#Full Spectrum Torch Dense
torch.linalg.eigh(H)

torch.return_types.linalg_eigh(
eigenvalues=tensor([-157.7768, -157.6267, -157.6045,  ...,   75.8858,   76.0343,
          76.0752], dtype=torch.float64),
eigenvectors=tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64))

In [9]:
#Partial Spectrum Torch Dense
torch.lobpcg(H, 2)

(tensor([76.0752, 76.0343], dtype=torch.float64),
 tensor([[-2.9532e-10, -1.9814e-10],
         [ 1.3182e-10,  2.0183e-11],
         [ 6.6980e-11, -2.7514e-10],
         ...,
         [-1.3886e-10,  1.1643e-10],
         [ 1.1206e-10,  5.7102e-11],
         [ 1.4057e-10, -2.5763e-10]], dtype=torch.float64))

In [10]:
#Partial Spectrum Torch Sparse
torch.lobpcg(H_sparse, 2)

(tensor([76.0752, 76.0343], dtype=torch.float64),
 tensor([[ 5.4486e-12, -2.7999e-10],
         [ 2.2276e-10, -7.7828e-11],
         [-3.1334e-11, -2.5219e-10],
         ...,
         [ 1.0696e-10,  6.4739e-11],
         [-1.9427e-10,  2.1133e-11],
         [ 1.9909e-11,  1.3727e-10]], dtype=torch.float64))

In [11]:
#Full Spectrum Scipy Dense
sp.linalg.eigh(H_numpy)

(array([-157.77681003, -157.6266625 , -157.604508  , ...,   75.88580993,
          76.03425181,   76.075244  ]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [12]:
#Partial Spectrum Scipy Dense - scipy sparse
sps.linalg.eigsh(H_numpy,2)


(array([-157.77681003, -157.6266625 ]),
 array([[ 1.03272517e-14, -4.92027684e-15],
        [ 1.00787400e-14, -4.35710706e-15],
        [ 9.81354312e-15, -3.82884838e-15],
        ...,
        [-3.93826972e-16,  6.28450985e-16],
        [-3.75799504e-16,  6.03405807e-16],
        [-3.41428055e-16,  5.49086119e-16]]))

In [13]:
#Partial Spectrum Scipy Sparse - scipy sparse
sps.linalg.eigsh(H_sparse_numpy,2)


(array([-157.77681003, -157.6266625 ]),
 array([[ 2.55792070e-15, -3.34409652e-15],
        [ 2.53985619e-15, -3.64202686e-15],
        [ 2.49580196e-15, -3.94268261e-15],
        ...,
        [-7.00167786e-17,  1.91134554e-16],
        [-1.20290108e-16,  2.69439318e-16],
        [-1.64388207e-16,  3.41727118e-16]]))

In [14]:
A = xitorch.LinearOperator.m(H)
xitorch.LinearOperator._getparamnames(A, "EJ, EJ")

[]

In [15]:
#Partial Spectrum xitorch Exact
val, vec = xitorch.linalg.symeig(A, 2, method="exacteig")

In [16]:
#Partial Spectrum  xitorch Davidson
val, vec = xitorch.linalg.symeig(A, 2, method="davidson", max_niter=100, nguess=None, v_init="randn", max_addition=None, min_eps=1e-06, verbose=False)

In [17]:
#Partial Spectrum  xitorch Davidson (1000 interations)
val, vec = xitorch.linalg.symeig(A, 2, method="davidson", max_niter=1000, nguess=None, v_init="randn", max_addition=None, min_eps=1e-06, verbose=False)

In [18]:
#Full Spectrum Jax Dense

#Partial Spectrum Jax Dense (lobpcg)

#Partial Spectrum Jax Sparse (dont know if this works)

#autograd eigensolve time



##### gradient computation times #######

#xitorch davidon (100 iterations) eigensolve with gradient time

#xitorch davidon (1000 iterations) eigensolve with gradient time

#quantum paper apraoch (if can figure out)

#autograd with wrapper (based off legume code)

#jax with wrapper (based off Jacks utils code)